In [1]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import transformers
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
!pip install datasets
!pip install evaluate
from datasets import Dataset
import evaluate
import scipy
from scipy.special import expit
#
print('torch version',torch.__version__)
print('pandas version',pd.__version__)
print('numpy version',np.__version__)
print('scipy version',transformers.__version__)
print(scipy.__version__)
input('WAIT')
drive.mount('/content/drive')
##
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#####PREPROCESSING####
#read file lines

with open('/content/drive/MyDrive/Data/BGL.log','r') as file:
  lines = file.readlines()#perno tis grammes tou arxeiou
  lines = [line.strip() for line in lines]#list comprehension gia an vgalo ta newlines
###
log_templates = pd.read_csv('/content/drive/MyDrive/Data/BGL_templates.csv')

##
part = r"(?P<Label>.+)\s(?P<Timestamp>\d{10})\s(?P<Date>\d{4}\.\d{2}\.\d{2})\s(?P<Node>\S+)\s(?P<Time>\S+)\s(?P<NodeRepeat>\S+)\s(?P<Type>\S+)\s(?P<Component>\S+)\s(?P<Level>\S+)"
regex = rf"{part}(\s(?P<Content>.+))?"
#to f gia antikatastasi {variable}
#
#list comprehension - vazo me ayth th texniki se mia lista to dictionary pou prokyptei apo ta matches kai tis antistixiseis tous sta groups
lines = [(re.match(regex,line)).groupdict() for line in lines]

print(lines[0:2])
#To dataframe
logs = pd.DataFrame.from_dict(lines)
#display(HTML((logs.sample(1)).to_html()))
####

#logs = pd.read_csv('/content/drive/MyDrive/Data/BGL_2k.log_structured.csv')

###CONVERT TO DATETIME####
logs['Time'] = pd.to_datetime(logs['Time'],format="%Y-%m-%d-%H.%M.%S.%f")
#drop useless columns
logs.drop(columns=['Timestamp','Date','NodeRepeat','Type','Component','Level'],inplace=True)
##NO NULL VALUES
logs = logs[(logs['Node'] != 'NULL') & (logs['Node'] != 'UNKNOWN_LOCATION')]
#

###SAMPLE EVERY NTH ROWS#####
print('Class Distribution before sampling: ',logs['Label'].value_counts())

logs = logs.iloc[::10]#px kathe 10 deigmata pare +1 deigma
#########
print('Class Distribution after sampling: ',logs['Label'].value_counts())
print('Nulls of Content',logs['Content'].isnull().sum())
print('Nulls of Time: ',logs['Time'].isnull().sum())
#
print('Nulls of Node: ',logs['Node'].isnull().sum())
#
logs['Node'] = logs['Node'].fillna('NULL')

#
print('Nulls of Node: ',logs['Node'].isnull().sum())

#
print('Lines where Node is NULL',logs['Node'].value_counts().get('NULL',0))
print('Lines where Node is UNKNOWN_LOCATION',logs['Node'].value_counts().get('UNKNOWN_LOCATION',0))
##MPORO NA RIKSO KAI TA LINES ME NULL NODES H UNKNOWN LOCATION EFOSON STO UNKNOWN LOCATION MPOREI
#NA ANAFERETAI SE DIAFORETIKA NODES EN TELEI ARA DEN SXETIZONTAI NO CONTEXT H KAI OXI
#print('Random null content row: ',(logs[logs['Content'].isnull()]).sample(1))
##
#display(HTML(((logs[logs['Content'].isnull()]).sample(1)).to_html()))
input('WAIT')
###Fill None Content with empty string or drop them(de tha to proteina-alla meionei ton ogko ton dedomenon)##
logs['Content'] = logs['Content'].fillna('')
##Drop unknown nodes or null nodes###
#
print('Lines where Node is NULL',logs['Node'].value_counts().get('NULL',0))
print('Lines where Node is UNKNOWN_LOCATION',logs['Node'].value_counts().get('UNKNOWN_LOCATION',0))
###
#####
##
##CLASS/LABELS ENCODING######
##
'''
encoder = LabelEncoder()
logs['Label'] = encoder.fit_transform(logs['Label'])
'''
#############################
logs.to_csv('/content/drive/MyDrive/Saves/logs.csv')
logs = logs.sort_values(by=['Node','Time']).groupby('Node',dropna=False)#sortare vash nodeid kai meta xronou oste kathe group na einai xronika sortarismeno kai meta groupare ta vash nodeID
#
ranges = logs.agg(MinTime=('Time', 'min'),MaxTime=('Time', 'max'))
ranges['TimeRangeDays'] = (ranges['MaxTime'] - ranges['MinTime']).dt.days
print('Mean duration of each node: ',ranges['TimeRangeDays'].mean())
##An deigmatolipto omoiomorfa ana 10 deigmata->meso duration / group = 70 meres
#an einai to bgl_2k -> xoris deigmatolipsia einai 1.6 meres / group meso oro
#an einai bgl_2k -> me deigmatolipsia 0.3 meres / group meso oro
#an einai kanoniko xoris deigmatolipsia >>> meres 170 kati / group meso oro
'''
_,first_group = list(logs)[0]
print(first_group.empty)
first_group = pd.DataFrame(first_group)
print(' duration of Node : R02-M1-N0-C:J12-U11',(first_group['Time'].max() - first_group['Time'].min()).days)
'''

#
input('WAIT')
##GROUPING IN FIXED TIME WINDOWS WITH NODEID AND WINDOW SIZE##
#windows = pd.DataFrame(columns=['Node','Window Start','text','labels'])#ena label column alla ekkremei pos tha ypologistei
WINDOW_SIZE = "1h" ####VARIABLE
ANOMALY_PERCENTAGE = 0.01
windows = []
#########
def create_time_windows(group):
  global windows

  #vale sto windows dataframe to id tou node os mia stili,allh stili to start tou window, sthlh text combined ola ta content tou xronikou group/>se deyterh fash to label
  #enose ola ta content tou time window se mia log paragraph
  #
  ##
  time_windows = group.groupby(pd.Grouper(key='Time',freq="1W",label='left'))#ftiakse ta parathyra aytou tou node
  #ena groupby object einai ena tuple ID-Data ego xrhsimopoio ta data
  for start,subgroup in time_windows:#ena grouby object epistrefei ena tuple tou typou (id,data) xrhsimopoio ta data kai to id paei axrisimopoihto
    #joinare ola ta strings tou subgroup kai valta sto neo dataframe
    text = (' '.join(subgroup['Content']).strip())#strip kostizei
    node_name = group.name
    #window_start = subgroup['Time'].min()
    window_start = start
    #
    #EXO PROVLIMA AN EXO ADEIA GROUPS->AXRISTA DEN PROSFEROUN PLIROFORIA-CLASSIFICATION-SYSXETISEIS->PIO POLLA MHDENIKA->PIO MEGALO IMBALANCE->XEIROTERH APODOSH->DIVISION/0 ERROR
    labels = 0
    if len(subgroup) == 0:
      continue#axristo group
    else:
      #anomaly_percentage = len(subgroup[subgroup['Label'] != '-']) / len(subgroup['Label'])
      #labels = 1 if anomaly_percentage > ANOMALY_PERCENTAGE else 0
      labels = 1 if len(subgroup[subgroup['Label'] != '-']) > 0 else 0 #1vs ALL

    data = {'Node': node_name, 'Window Start': window_start,'text': text,'labels':labels}  # ta [] ta thelei alios peta error kati gia scalar values
    #windows = pd.concat([windows, data]) -->>OXIIII PIANEI PARA POLLH MNHMH MH APODOTIKO
    #vale os window start h to key tou subgroup h to min ton datetimes tou group
    #vale os nodeid to group.name
    windows.append(data)
  #
  ##
  return
######APPLY THE FUNCTION TO THE LOGS#######
######TO PARATHYRO PREPEI NA PAREI KAI KAPOIO LABEL MPORO PX ME DIAFOROUS TROPOUS
####EPEIDH EINAI POLLA TA BENIGN GIA KALYTERO BALANCE THA MPOROUSA PX AKOMA KAI
###ENA ALLO LABEL NA YPARXEI STO GROUP PERA TOU BENIGN NA TO THEORISO ANOMALY
###H ME KAPOIO THRESHOLD TON MH BENIGN KAI PANO NA THEOREITAI ANOMALY
logs.apply(create_time_windows)
###
windows = pd.DataFrame(windows)
windows.info()
print('Distribution of classes',windows['labels'].value_counts())
##SAVING TO CSV#

windows.to_csv('/content/drive/MyDrive/Saves/windows.csv')
##


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00
torch version 2.5.1+cu121
pandas version 2.2.2
numpy version 1.26.4
scipy version 4.

KeyboardInterrupt: Interrupted by user